 # using pre-trained word embeddings, IMDB sentiment detection data base
 
 
 
 word emmbeding: Word2Ves,
 
 dl link IMDB raw_database: https://ai.stanford.edu/~amaas/data/sentiment/ 
 
 dl link e GloVe word embeding: nlp.stanford.edu/projects/glove/ 
 
 2 class (neg and pos),
 25000 train data (12500 each class),
 25000 test data (12500 each class),
 common words pas feature size =10000

In [23]:
import os 
import numpy as np

imdb_dir='./datasets/Imdb'
train_dir=os.path.join(imdb_dir, 'train')

texts = [] #liste text ha
labels = [] # liste label ha


for label_type in ['neg', 'pos']:
    dir_name=os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name): 
        # for fname in directory -->  read each file 
        if fname[-4:] == '.txt': # if  file format is .txt 
            f = open (os.path.join(dir_name , fname), errors="ignore") 
            texts.append(f.read()) 
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)           

In [24]:
# tokenize and vectorize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

training_samples=12500 # 
validation_samples=10000 # 5000 each class

maxlen=100 
max_words=10000 # N-most common words
embedding_dim = 50 

tokenizer =Tokenizer (num_words = max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' %len(word_index)) # Found 88584 unique tokens.


data = pad_sequences(sequences, maxlen=maxlen)

labels =np.asarray(labels) # 
print('Shape of data tensor:' , data.shape) #data.shape=(25000,100)
print('Shape of label tensor:' , labels.shape) #labels.shape= (25000,)

# shuffleing
indices = np.arange(data.shape[0]) #data.shape[0]=25000
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
# end shuffeling


x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data  [training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 88584 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [ ]:
# khandane file glove
# read  file glove r
glove_dir='./datasets/glove.6B'

embeddings_index = {}
f=open(os.path.join(glove_dir , 'glove.6b.50d.txt')) #50d har kalame be bordare 50 tai map shode
i=0
for line in f: # select line by line in each file
    i+=1
    values=line.split() 
    word = values[0] 
    embeddings_index[word]= np.asarray(values[1:],dtype='float32') 

f.close()

In [3]:
line

'hoأ\xa0ng -0.021476 -0.46788 -0.57489 0.25384 -0.83486 -1.2622 0.74499 0.41843 0.15076 -0.3762 0.31586 1.0779 0.26601 -0.84584 -0.21545 -0.30827 -1.0899 -0.53668 2.0235 1.3613 -0.9779 -0.80162 0.68561 -0.18649 -0.5215 0.8261 -0.1142 -0.37842 -0.70749 -0.082986 -0.54944 -0.25965 0.39062 1.385 0.35108 0.75802 0.064603 1.0825 -0.023899 -0.59102 0.16235 -0.25107 0.20679 -0.12571 -0.018305 -0.91191 -0.27518 -1.3942 0.9766 -0.11565\n'

In [30]:

embedding_dim = 50 

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items(): # word_index = tokenizer.word_index, dictionary word:index 
    if i < max_words: #max_words=10000
        embedding_vector = embeddings_index.get(word) # dic.get(index)=value=dic[index]
        
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector         

In [41]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

#freezing e embedding layer:
model.layers[0].set_weights([embedding_matrix]) # layers[0] = Embedding layer,  freez 
model.layers[0].trainable= False # freeze the embedding layer in training

In [ ]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model_history = model.fit(x_train, y_train,
                          epochs=10, batch_size=32,
                          validation_data=[x_val, y_val])

model.save_weights('pre_trained_glove_model.h5')

In [ ]:
# evaluation on test:

test_dir = os.path.join(imdb_dir, 'test')

texts = [] 
labels = [] 


for label_type in ['neg', 'pos']:
    dir_name=os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)): 
        
        if fname[-4:] == '.txt': 
            f = open (os.path.join(dir_name , fname), errors="ignore") 
            texts.append(f.read()) 
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

            
sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)
model.load_weights('pre_trained_golve_model.h5')
model.evaluate(x_test, y_test)


# RNN, LSTM

In [65]:
timesteps = 100
input_features = 32
output_features = 64

from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

model=Sequential()
model.add(Embedding(10000,32)) #9999-most common words, 
model.add(SimpleRNN(64, return_sequences=True)) 
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32)) # 
model.add(Dense(1, activation='sigmoid'))

In [67]:
from keras.layers import LSTM

model=Sequential()
model.add(Embedding(10000,32))
model.add(LSTM(32)) # 32 tedade neuronha
model.add(Dense(1, activation='sigmoid'))